# 01-05: Minimizzazione con SciPy

**5. I dati qui a fianco rappresentano un campione di 1000 valori indipendenti ottenuti con la formula a*rand()+b*rand(), a partire da due numeri reali a e b incogniti.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
base = pd.read_excel("inml25tst01.xlsx", sheet_name="Es 5", header=None)
base.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,5. I dati qui a fianco rappresentano un campio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.1,26.0,10.6,20.2,7.7,39.8,31.9,14.3,36.2,38.1


In [3]:
df = base.drop(columns=base.columns[:9])
print(df.head(2))
print(df.shape)

samples = df.values.flatten()
print(samples.shape)
print(samples[0])
print(samples[10])
print(samples.mean())

     9     10    11    12    13    14    15    16    17    18
0  25.1  26.0  10.6  20.2   7.7  39.8  31.9  14.3  36.2  38.1
1  25.4  21.3  18.8  15.0  21.1  34.1  48.9  32.2  26.2  10.2
(100, 10)
(1000,)
25.1
25.4
25.112599999999997


5.1 Si trovino due numeri reali s e t tali che:
- s rende minima la media di $|X_i-s|$ e,
- t rende minima la media di $(X_i-t)^2$

(Entrambe le medie si considerano fatte sul campione, al variare di i da 1 a 1000.)
Cos'altro rappresentano questi valori?

## Minimizzazione

`scipy.optimize.minimize` si usa per trovare il **minimo di una funzione scalare** rispetto a uno o più parametri.

Si parte definendo una **funzione obiettivo** che prende in input un vettore di parametri e restituisce un numero reale. Anche nel caso di un solo parametro, l’argomento è sempre un array.

```python
from scipy.optimize import minimize

f = lambda x: (x[0] - 2)**2
````

Poi si fornisce una **stima iniziale** (`x0`) e, se necessario, si specifica il **metodo di ottimizzazione**:

```python
res = minimize(f, x0=[0.0], method="BFGS")
```

Il risultato è un oggetto `OptimizeResult` che contiene le informazioni sull’ottimizzazione. In particolare:

* `res.x` è il punto in cui la funzione raggiunge il minimo
* `res.fun` è il valore minimo della funzione
* `res.success` indica se l’algoritmo è convergito correttamente

Per accedere al minimizzatore stimato:

```python
res.x[0]
```

La scelta del metodo è importante. Per funzioni **derivabili** (ad esempio quadratiche) sono adatti metodi come `BFGS`; per funzioni **non derivabili** (come il valore assoluto) è preferibile usare metodi come `Powell` o `Nelder-Mead`.

In sintesi, `minimize` permette di risolvere problemi di stima della forma:

$\hat{\theta} = \arg\min_\theta f(\theta)$

In [4]:
import numpy as np
from scipy.optimize import minimize

**Nota.** Si potrebbe scrivere anche `s` e `t` semplicemente, ma è più corretto scrivere `s[0]` perchè internamente `minimize` usa un vettore e non uno scalare, ma non cambia nulla

In [5]:
s_fun = lambda s: np.mean(np.abs(samples - s[0]))
t_fun = lambda t: np.mean((samples - t[0])**2)

s_min = minimize(s_fun, x0=samples[0])
t_min = minimize(t_fun, x0=samples[0])

s = s_min.x[0]
t = t_min.x[0]

print(f"s: {s:.4f}")
print(f"t: {t:.5f}")


s: 25.0000
t: 25.11260


$s$ è il valore che rende minima la deviazione assoluta media dal campione: coincide con la mediana campionaria ed è una misura di posizione robusta, poco sensibile agli outlier.

$t$ è il valore che rende minimo l’errore quadratico medio: coincide con la media campionaria ed è la misura di posizione “classica”, ottimale in senso dei minimi quadrati ma sensibile ai valori estremi.